In [59]:
import requests, os
from bs4 import BeautifulSoup
from src.downloader import download

def get_html(url):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text
    

def get_image(html):
    """Scrape share image."""
    image = []
    if html.find("meta", property="image"):
        image = html.find("meta", property="image").get('content')
    elif html.find("meta", property="og:image"):
        image = html.find("meta", property="og:image").get('content')
    elif html.find("meta", property="twitter:image"):
        image = html.find("meta", property="twitter:image").get('content')
    elif html.find("img", src=True):
        for img in html.find_all("img"):
            image.append(img.get('src'))
    return image


def check_word(word):
    def check(url):
        return (word in url)
    return check

In [14]:
keyword = "gadget electronique"
page = "2"
url = "https://www.amazon.fr/s?k={}&page={}&__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=sr_pg_{}".format(keyword, page, page)

soup = BeautifulSoup(get_html(url))
# print(soup.prettify())
for image_url in list(filter(check_word("_AC_UL320_"), get_image(soup))):
    download(image_url, "imgs")

SSLError: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/61YC5v2tk7L._AC_UL320_.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1076)')))

In [2]:
import os, requests, time, shutil, glob, bz2, imgkit, re
import numpy as np
from PIL import Image
from multiprocessing.dummy import Pool
from src.azurekey import *
from src.downloader import download

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
}

def search(search_term, azureKey, offset=0):
    # print('searching using bing: "'+search_term+'"')
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    subscription_key = azureKey
    assert subscription_key
    headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
    params  = {"q": search_term, "responseFilter": "Webpages","count":50, "offset":offset, "safeSearch":"Strict"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    return [v["url"] for v in search_results["webPages"]["value"]]

def clean(html):
    html_temp = re.sub('href=".*?"', 'href=""', html)
    html_temp = re.sub('src=".*?"', 'src=""', html_temp)
    html_temp = re.sub('<script.*?/script>', '', html_temp)
    return html_temp

def save_html2jpg(url, dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    req = requests.get(url, headers=headers)
    imgkit.from_string(clean(req.text), os.path.join(dir_name, str(hash(url))+".jpg"))


# def download(links_in, dir_name, n=None, log=set()):
#     links = links_in[:n]
#     # print('search results',len(links)
#     if not os.path.exists(dir_name):
#         os.mkdir(dir_name)

#     def fetch(url, log=log):
#         r=requests.get(url[0], stream=True)
#         r.raw.decode_content = True # handle spurious Content-Encoding
#         img = Image.open(r.raw)
#         bits = str(imagehash.average_hash(img))
#         if bits not in log:
#             log |= set([bits])
#         if len(log) % 100 == 0:
#             print("{} images saved.".format(len(log)))
#         img_file = dir_name+'/commercial_product_'+bits+".jpg"
#         img.save(img_file)
        

#     modLinks=[[links[i],i+1] for i in range(len(links))]
#     Pool(10).map(fetch, modLinks)

#     return log

In [16]:
search_results = search("electronics", azureKey, offset=0)

In [121]:
url = "https://www.google.com/"
req = requests.get(url, headers=headers)
# imgkit.from_url(url, os.path.join("webpages", str(hash(url))+".jpg"))

In [122]:
def repair_link(html, url):
#     html_temp = re.sub('href="/', 'href="{}'.format(url), html)
#     html_temp = re.sub('src="/', 'src="{}'.format(url), html_temp)
#     html_temp = re.sub("url\(/", "url({}".format(url), html_temp)
#     html_temp = re.sub('poster="/', 'poster="{}'.format(url), html_temp)
#     html_temp = re.sub('srcset="/', 'srcset="{}'.format(url), html_temp)
    html_temp = re.sub('"/([^/"][^<"]*?)"', r'"{}\1"'.format(url), html)
    html_temp = re.sub("'/([^/'][^<']*?)'", r'"{}\1"'.format(url), html_temp)
    html_temp = re.sub("url\(/", "url({}".format(url), html_temp)
    return html_temp

In [123]:
with open("webpages/test.html", "w") as f:
#     f.write(req.text)
    f.write(repair_link(req.text, url))

In [142]:
re.findall('[^ \'()"]*?\.png', req.text)

['/images/branding/googleg/1x/googleg_standard_color_128dp.png',
 '//ssl.gstatic.com/gb/images/i1_1967ca6a.png',
 '//ssl.gstatic.com/gb/images/i2_2ec824b0.png',
 '/images/experiments/wavy-underline.png',
 '/images/searchbox/desktop_searchbox_sprites302_hr.png',
 '/images/searchbox/desktop_searchbox_sprites302_hr.png',
 '/images/searchbox/desktop_searchbox_sprites302_hr.png',
 '/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png',
 '/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png',
 '/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png',
 'https://ssl.gstatic.com/gb/images/p2_8e348e57.png',
 '//ssl.gstatic.com/gb/images/i1_1967ca6a.png',
 '//ssl.gstatic.com/gb/images/i2_2ec824b0.png',
 '/images/nav_logo299.png']

In [188]:
def download(url, temp_dir, webpage_dir):
    save_path = webpage_dir + "/" + temp_dir
    r = requests.get(url, allow_redirects=True, headers=headers)
    file_name = os.path.split(url)[-1]
    print(file_name)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_file_path = save_path + "/" + file_name
    open(save_file_path, 'wb').write(r.content)
    return temp_dir + "/" + file_name

def download_all(link_list, url, log, temp_dir="temp", webpage_dir="webpages"):
    for link in link_list:
        if link in log:
            continue
        if link.startswith("http"):
            log[link] = download(link, temp_dir, webpage_dir)
        elif link.startswith("//"):
            try:
                log[link] = download("https:" + link, temp_dir, webpage_dir)
            except:
                pass
        elif link.startswith("/"):
            link_temp = url+link[1:]
            log[link] = download(link_temp, temp_dir, webpage_dir)
    return log

def replace_all(html, log):
    html_temp = html
    for e, v in log.items():
        html_temp = re.sub(e, v, html_temp)
    return html_temp

In [189]:
url = "https://www.accenture.com/"
req = requests.get(url, headers=headers)

In [190]:
log = {}
log = download_all(re.findall('[^ \'()"]*?\.png', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.jpg', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.js', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.css', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.json', req.text), url, log, temp_dir="temp")

Accenture-Living-Business-Twitter.png
Accenture-Homepage-Image-400x400.jpg
Accenture-Home-Social-Image.jpg
France.jpg
Accenture-we-act-for-impact-1920x1080.jpg
70a2765e-e451-4bb1-83dd-6e13d136a97f.js
otSDKStub.js
wDq4Hx70.min.js
launch-EN664f8f34ad5946f8a0f7914005f717cf.min.js
ip.js
embed.js
ScrollMagic.min.js
debug.addIndicators.min.js
gsap.min.js
TweenLite.min.js
TweenMax.min.js
slick.js
ionicons.js
quick-fix-js.js
quick-fix-css.css
slick.css
slick-theme.css
ip.json


In [191]:
with open("webpages/test.html", "w") as f:
    f.write(replace_all(req.text, log))

In [19]:
r

<Response [403]>

In [20]:
search_url = "https://api.apiflash.com/v1/urltoimage?access_key=37bc0ab0376f4a33b4bac2e7a47776c4&format=jpeg&height=1920&js=(function%20(global%2C%20factory)%20%7B%0A%20%20%20%20typeof%20exports%20%3D%3D%3D%20'object'%20%26%26%20typeof%20module%20!%3D%3D%20'undefined'%20%3F%20module.exports%20%3D%20factory()%20%3A%0A%20%20%20%20typeof%20define%20%3D%3D%3D%20'function'%20%26%26%20define.amd%20%3F%20define(factory)%20%3A%0A%20%20%20%20(global%20%3D%20typeof%20globalThis%20!%3D%3D%20'undefined'%20%3F%20globalThis%20%3A%20global%20%7C%7C%20self%2C%20global.blurify%20%3D%20factory())%3B%0A%7D(this%2C%20(function%20()%20%7B%20'use%20strict'%3B%0A%0A%20%20%20%20function%20preload(images)%20%7B%0A%20%20%20%20%20%20%20%20var%20imageNodes%20%3D%20%5B%5D%3B%0A%20%20%20%20%20%20%20%20var%20count%20%3D%200%2C%20doneAction%20%3D%20function%20(images)%20%7B%20%7D%3B%0A%20%20%20%20%20%20%20%20images%20%3D%20(typeof%20images%20!%3D%20'object')%20%3F%20%5Bimages%5D%20%3A%20images%3B%0A%20%20%20%20%20%20%20%20images.length%20%3D%3D%3D%200%20%26%26%20doneAction(imageNodes)%3B%0A%20%20%20%20%20%20%20%20images.map(function%20(image)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20var%20_image%20%3D%20new%20Image()%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20_image.crossOrigin%20%3D%20'*'%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20console.log(image.dataset.src)%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20_image.src%20%3D%20image.dataset%20%3F%20image.dataset.src%20%3A%20image.getAttribute('src')%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20_image.onload%20%3D%20imageLoad%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20_image.onerror%20%3D%20imageLoad%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20imageNodes.push(_image)%3B%0A%20%20%20%20%20%20%20%20%7D)%3B%0A%20%20%20%20%20%20%20%20function%20imageLoad()%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20count%2B%2B%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20if%20(count%20%3D%3D%20images.length)%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20doneAction(imageNodes)%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20return%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20done%3A%20function%20(cb)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20doneAction%20%3D%20arguments%5B0%5D%20%7C%7C%20doneAction%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D%2C%0A%20%20%20%20%20%20%20%20%7D%3B%0A%20%20%20%20%7D%0A%20%20%20%20function%20cssSupport(key%2C%20value)%20%7B%0A%20%20%20%20%20%20%20%20var%20element%20%3D%20document.createElement('div')%3B%0A%20%20%20%20%20%20%20%20switch%20(arguments.length)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20case%201%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20return%20key%20in%20element.style%20%3F%20true%20%3A%20false%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20case%202%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20element.style%5Bkey%5D%20%3D%20value%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20return%20element.style%5Bkey%5D%20%3F%20true%20%3A%20false%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20default%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20return%20false%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%0A%20%20%20%20function%20blurify(options)%20%7B%0A%20%20%20%20%20%20%20%20if%20(options%20%3D%3D%3D%20void%200)%20%7B%20options%20%3D%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20blur%3A%206%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20mode%3A%20'auto'%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20images%3A%20%5B%5D%2C%0A%20%20%20%20%20%20%20%20%7D%3B%20%7D%0A%20%20%20%20%20%20%20%20if%20(!(this%20instanceof%20blurify))%0A%20%20%20%20%20%20%20%20%20%20%20%20return%20new%20blurify(options)%3B%0A%20%20%20%20%20%20%20%20if%20(typeof%20options%20%3D%3D%3D%20'number')%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20options%20%3D%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20blur%3A%20options%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20images%3A%20arguments%5B1%5D%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20mode%3A%20'auto'%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20this.options%20%3D%20options%3B%0A%20%20%20%20%20%20%20%20this.blur%20%3D%20options.blur%20%7C%7C%206%3B%0A%20%20%20%20%20%20%20%20this.mode%20%3D%20options.mode%20%7C%7C%20'css'%3B%0A%20%20%20%20%20%20%20%20%2F%2F%20%40ts-ignore%0A%20%20%20%20%20%20%20%20console.log(options.images)%3B%0A%20%20%20%20%20%20%20%20this.%24els%20%3D%20options.images.nodeType%20%3D%3D%201%20%3F%20%5Boptions.images%5D%20%3A%20%5B%5D.slice.call(options.images)%3B%0A%20%20%20%20%20%20%20%20if%20(this.mode%20%3D%3D%20'auto')%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20cssSupport('filter'%2C%20'blur(1px)')%20%3F%20this.useCSSMode()%20%3A%20this.useCanvasMode()%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20else%20if%20(this.mode%20%3D%3D%20'css')%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20this.blur%20%3D%20this.blur%20%2F%202%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20this.useCSSMode()%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20else%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20this.useCanvasMode()%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%20%20blurify.prototype.useCSSMode%20%3D%20function%20()%20%7B%0A%20%20%20%20%20%20%20%20var%20_this%20%3D%20this%3B%0A%20%20%20%20%20%20%20%20console.log(this.%24els)%3B%0A%20%20%20%20%20%20%20%20this.%24els.map(function%20(el)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20console.log('el'%2C%20el)%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%2F%2F%20el.src%20%3D%20el.dataset%20%3F%20el.dataset.src%20%3A%20el.getAttribute('src')%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20el.style%5B'filter'%5D%20%3D%20el.style%5B'-webkit-filter'%5D%20%3D%20%22blur(%22%20%2B%20_this.blur%20%2B%20%22px)%22%3B%0A%20%20%20%20%20%20%20%20%7D)%3B%0A%20%20%20%20%7D%3B%0A%20%20%20%20blurify.prototype.useCanvasMode%20%3D%20function%20()%20%7B%0A%20%20%20%20%20%20%20%20var%20_this%20%3D%20this%3B%0A%20%20%20%20%20%20%20%20this.imageType%20%3D%20this.options.imageType%20%7C%7C%20%22image%2Fjpeg%22%3B%0A%20%20%20%20%20%20%20%20preload(this.%24els).done(function%20(images)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20images.map(function%20(image%2C%20index)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20_this.%24els%5Bindex%5D.src%20%3D%20_this.getDataURL(image)%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D)%3B%0A%20%20%20%20%20%20%20%20%7D)%3B%0A%20%20%20%20%7D%3B%0A%20%20%20%20blurify.prototype.blurify%20%3D%20function%20(canvas%2C%20blur)%20%7B%0A%20%20%20%20%20%20%20%20var%20ctx%20%3D%20canvas.getContext('2d')%3B%0A%20%20%20%20%20%20%20%20ctx.globalAlpha%20%3D%201%20%2F%20(2%20*%20%2Bblur)%3B%0A%20%20%20%20%20%20%20%20for%20(var%20y%20%3D%20-blur%3B%20y%20%3C%3D%20blur%3B%20y%20%2B%3D%202)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20for%20(var%20x%20%3D%20-blur%3B%20x%20%3C%3D%20blur%3B%20x%20%2B%3D%202)%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20ctx.drawImage(canvas%2C%20x%2C%20y)%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20if%20(x%20%3E%3D%200%20%26%26%20y%20%3E%3D%200)%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20ctx.drawImage(canvas%2C%20-(x%20-%201)%2C%20-(y%20-%201))%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20ctx.globalAlpha%20%3D%201%3B%0A%20%20%20%20%7D%3B%0A%20%20%20%20blurify.prototype.getDataURL%20%3D%20function%20(image)%20%7B%0A%20%20%20%20%20%20%20%20var%20canvas%20%3D%20document.createElement('canvas')%3B%0A%20%20%20%20%20%20%20%20var%20ctx%20%3D%20canvas.getContext('2d')%3B%0A%20%20%20%20%20%20%20%20canvas.width%20%3D%20image.width%3B%0A%20%20%20%20%20%20%20%20canvas.height%20%3D%20image.height%3B%0A%20%20%20%20%20%20%20%20ctx.drawImage(image%2C%200%2C%200)%3B%0A%20%20%20%20%20%20%20%20this.blurify(canvas%2C%20this.blur)%3B%0A%20%20%20%20%20%20%20%20return%20canvas.toDataURL(this.imageType)%3B%0A%20%20%20%20%7D%3B%0A%0A%20%20%20%20return%20blurify%3B%0A%0A%7D)))%3B%0A%2F%2F%23%20sourceMappingURL%3Dblurify.js.map%0A(function%20()%20%7B%0A%20%20%20%20new%20blurify(%7B%0A%20%20%20%20%20%20%20%20images%3A%20document.querySelectorAll('img')%2C%0A%20%20%20%20%20%20%20%20blur%3A%2030%2C%0A%20%20%20%20%20%20%20%20mode%3A%20'css'%2C%0A%20%20%20%20%7D)%3B%0A%7D)()%3B&response_type=image&url=https%3A%2F%2Fwww.google.com%2F"




In [16]:
import urllib.parse
urllib.parse.quote('<script src="http://35.246.157.75:5000/static/js/blurify.js"></script>'.encode('utf-8'))

'%3Cscript%20src%3D%22http%3A//35.246.157.75%3A5000/static/js/blurify.js%22%3E%3C/script%3E'

In [21]:
dir_name = "webpages"
url = "https://www.google.com/"
# search_url = "https://api.apiflash.com/v1/urltoimage?access_key=37bc0ab0376f4a33b4bac2e7a47776c4&height=1920&js=http%3A%2F%2F35.246.157.75%3A5000%2Fstatic%2Fjs%2Fblurify.js&url={}".format(url)
r=requests.get(search_url, stream=True)
r.raw.decode_content = True # handle spurious Content-Encoding
img = Image.open(r.raw)
img_file = os.path.join(dir_name, str(hash(url))+".jpg")
img.save(img_file)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f9f23fc76b0>

In [ ]:
http://35.246.157.75:5000%2Fstatic%2Fjs%2Fblurify.js
http%3A%2F%2F35.246.157.75%3A5000%2Fstatic%2Fjs%2Fblurify.js